In [ ]:
#cons
MODEL_FOR_TRANSFORMER="sentence-transformers/all-MiniLM-L6-v2"
!pip install scikit-learn
!pip install pandas
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

#function read file - to be changed for cloud
def readFile():
  uploaded = files.upload()
  uploaded_file_path = list(uploaded.keys())[0]
  dfImportedResults = pd.read_csv(uploaded_file_path)
  return dfImportedResults



In [ ]:
#call the dataframe imported

df=readFile()
#print(df)


In [ ]:

print(df.head())

In [ ]:


# Load the transformer model and tokenizer
model_name = MODEL_FOR_TRANSFORMER
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to get sentence embeddings
def get_sentence_embedding(sentence):
    tokens = tokenizer(sentence, return_tensors="pt")
    outputs = model(**tokens)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy()

# Function to calculate cosine similarity between a reference cell and two other cells
def calculate_cosine_similarity(reference, target1, target2):
    embedding_ref = get_sentence_embedding(reference).reshape(1, -1)
    embedding_target1 = get_sentence_embedding(target1).reshape(1, -1)
    embedding_target2 = get_sentence_embedding(target2).reshape(1, -1)

    similarity_score1 = cosine_similarity(embedding_ref, embedding_target1)[0][0]
    similarity_score2 = cosine_similarity(embedding_ref, embedding_target2)[0][0]

    return similarity_score1, similarity_score2

# Calculate cosine similarity for each pair of cells using a loop
for index in range(len(df)):
    reference_answer = df.at[index, 'answer']
    target1_gemini = df.at[index, 'geminiRep1']
    target2_gpt35 = df.at[index, 'gpt35Rep1']

    similarity_score1, similarity_score2 = calculate_cosine_similarity(reference_answer, target1_gemini, target2_gpt35)

    # Create new columns for cosine similarity scores
    df.at[index, 'Cosine_Similarity_Target1Gemini'] = similarity_score1
    df.at[index, 'LabelGemini'] = (similarity_score1 > 0.5).astype(int)
    df.at[index, 'Cosine_Similarity_Target2GPT'] = similarity_score2
    df.at[index, 'LabelGPT'] = (similarity_score2 > 0.5).astype(int)
# Display the updated DataFrame
print(df)


In [ ]:
csv_filename = 'FinalSimilarity.csv'
df.to_csv(csv_filename, index=False)